# Eigenvalues and Eigenvectors, Matplotlib, and Univariate Time Series

<div align="right"><button><a href="https://colab.research.google.com/github/QuantEcon/workshop.africa-july2023/blob/main/day-06/exercise_set_6_with_solution.ipynb"><img src="" heght="10px"/><img
  src="https://colab.research.google.com/assets/colab-badge.svg"
  alt="open with Colab" width="100px"/></a></button></div>

#### Written for the QuantEcon Africa Workshop (July 2023)
#### Author: [Humphrey Yang](https://github.com/HumphreyYang)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eig
from matplotlib import cm

## Exercise 1

***Exercise 1.1*** Given a matrix $A$, compute the eigenvalues, eigenvectors and plot them.

In [ ]:
A = np.array([[1, 4], 
              [5, 7]])

### Solution

In [ ]:
A = np.array(A)
evals, evecs = eig(A)
evals = evals.real
evecs = evecs[:, 0], evecs[:, 1]

print("Eigenvalues:", evals)
print("Eigenvectors:", evecs)

fig, ax = plt.subplots(figsize=(7, 5))

# Set the axes through the origin
for spine in ['left', 'bottom']:
    ax.spines[spine].set_position('zero')
for spine in ['right', 'top']:
    ax.spines[spine].set_color('none')

ax.grid(alpha=0.4)

xmin, xmax = -3, 3
ymin, ymax = -3, 3
ax.set(xlim=(xmin, xmax), ylim=(ymin, ymax))

# Plot each eigenvector
for v in evecs:
    ax.annotate('', xy=v, xytext=(0, 0),
                arrowprops=dict(facecolor='blue',
                shrink=0,
                alpha=0.6,
                width=0.5))

# Plot the image of each eigenvector
for v in evecs:
    z = A @ v
    ax.annotate('', xy=z, xytext=(0, 0),
                arrowprops=dict(facecolor='red',
                shrink=0,
                alpha=0.6,
                width=0.5))

# Plot the lines they run through
x = np.linspace(xmin, xmax, 3)
for v in evecs:
    a = v[1] / v[0]
    ax.plot(x, a * x, 'b-', lw=0.4)

plt.show()

***Exercise 1.2*** Given a matrix $A$, compute the eigenvalues, eigenvectors of the inverse of $A$ ($A^{-1}$). Compare it to the eigenvalue of $A$.

(Hint: Try to compute 1/eigenvalue of $A$ and compare it to the eigenvalue of $A^{-1}$)

### Solution

In [ ]:
eig(np.linalg.inv(A))

In [ ]:
eig(A)

In [ ]:
np.allclose(1/eig(A)[0], eig(np.linalg.inv(A))[0])

***Exercise 1.3*** Try to come up with a simple proof of the relationship between the eigenvalues of $A$ and $A^{-1}$ you found in Exercise 1.2 (Think about what conditions are needed for the proof to hold)


### Solution

We start with the definition of eigenvalues:
$$ 
A v = \lambda\ v
$$

Then we multiply both sides by $A^{-1}$:

$$
A^{-1}A v  = \lambda A^{-1} v
$$  

Since $A^{-1}A = I$, we have:

$$
\frac{1}{\lambda} v = A^{-1} v
$$

## Exercise 2

Compute the spectral radius of the matrix A.

### Solution

In [ ]:
A = np.array([[1, 4], 
              [5, 7]])

evals, evecs = eig(A)   # finding eigenvalues and eigenvectors

r = max(abs(evals))    # compute spectral radius
print(r)

## Exercise 3

Without using the `eig` function, compute the dominant eigenvalue and its corresponding eigenvector of the matrix $A$ in Question 3.1.

(Hint: you can use [power iteration method](https://en.wikipedia.org/wiki/Power_iteration) to solve this problem.)

### Solution

In [ ]:
A = np.array([[1, 4], 
              [5, 7]])

def power_iteration(A, num_iterations=1000):
    n = A.shape[0]
    b_k = np.ones(n)
    for _ in range(num_iterations):
        b_k1 = A @ b_k
        b_k1_norm = np.linalg.norm(b_k1)
        b_k = b_k1 / b_k1_norm
    eigenvalue = b_k @ A @ b_k  # Rayleigh quotient
    return eigenvalue, b_k

eigenvalue, eigenvector = power_iteration(A)
print("Dominant Eigenvalue: \n", eigenvalue)
print("Corresponding Eigenvector: \n", np.abs(eigenvector))

# Compare your result to the output of `np.linalg.eig`:
A = np.array(A)
evals, evecs = eig(A)
evals = evals.real
evecs = evecs[:, 0], evecs[:, 1]

print("\nEigenvalues (numpy): \n", evals)
print("Eigenvectors (numpy): \n", np.abs(evecs))


For this simple matrix, the result is very accurate.

## Exercise 4

[Wassily Leontief](https://en.wikipedia.org/wiki/Wassily_Leontief) developed a model of an economy with $n$ sectors producing $n$ different commodities representing the interdependencies of different sectors of an economy.

Under this model some of the output is consumed internally by the industries and the rest is consumed by external consumers.

We define a simple model with 3 sectors - agriculture, industry, and service.

The following table describes how output is distributed within the economy:

|             | Total output | Agriculture | Industry | Service | Consumer |
|:-----------:|:------------:|:-----------:|:--------:|:-------:|:--------:|
| Agriculture |     $x_1$    |   0.3 $x_1$  | 0.2 $x_2$ |0.3 $x_3$ |     4    |
|   Industry  |     $x_2$    |   0.2 $x_1$  | 0.4 $x_2$ |0.3 $x_3$ |     5    |
|   Service   |     $x_3$    |   0.2 $x_1$  | 0.5 $x_2$ |0.1 $x_3$ |    12    |

The first row depicts how agriculture's total output $x_1$ is distributed

* $0.3x_1$ is used as inputs within agriculture itself,
* $0.2x_2$ is used as inputs by the industry sector to produce $x_2$ units,
* $0.3x_3$ is used as inputs by the service sector to produce $x_3$ units and
* 4 units is the external demand by consumers.

We can transform this into a system of linear equations for the 3 sectors as
given below:

$$
    x_1 = 0.3x_1 + 0.2x_2 + 0.3x_3 + 4 \\
    x_2 = 0.2x_1 + 0.4x_2 + 0.3x_3 + 5 \\
    x_3 = 0.2x_1 + 0.5x_2 + 0.1x_3 + 12
$$

This can be transformed into the matrix equation $x = Ax + d$ where

$$
x =
\begin{bmatrix}
    x_1 \\
    x_2 \\
    x_3
\end{bmatrix}
, \; A =
\begin{bmatrix}
    0.3 & 0.2 & 0.3 \\
    0.2 & 0.4 & 0.3 \\
    0.2 & 0.5 & 0.1
\end{bmatrix}
\; \text{and} \;
d =
\begin{bmatrix}
    4 \\
    5 \\
    12
\end{bmatrix}
$$

The solution $x^{*}$ is given by the equation $x^{*} = (I-A)^{-1} d$

***Exercise 4.1*** Find the spectral radius of $A$ and verify if the spectral radius is less than 1.

### Solution

In [ ]:
A = np.array([[0.3, 0.2, 0.3],
              [0.2, 0.4, 0.3],
              [0.2, 0.5, 0.1]])

evals, evecs = eig(A)

r = max(abs(evals))   #dominant eigenvalue/spectral radius
print(r)

***Exercise 4.2*** Use the Neumann Series Lemma to find the solution $x^{*}$ if it exists.

### Solution

In [ ]:
I = np.identity(3)
B = I - A

d = np.array([4, 5, 12])
d.shape = (3,1)

B_inv = np.linalg.inv(B)
x_star = B_inv @ d
print(x_star)

## Exercise 5

As an exercise, we ask you to represent and solve a third order linear difference equation. How many initial conditions must you specify?

### Solution

$$
\begin{bmatrix} 
1 & 0 & 0 & 0 &\cdots & 0 & 0 & 0 \cr
-\lambda_1 & 1 & 0 & 0 &\cdots & 0 & 0 & 0 \cr
-\lambda_2 & -\lambda_1 & 1 & 0 & \cdots & 0 & 0 & 0 \cr
-\lambda_3 & -\lambda_2 & -\lambda_1 & 1 & \cdots & 0 & 0 & 0 \cr
 \vdots & \vdots & \vdots & \vdots & \cdots & \vdots & \vdots & \vdots \cr
0 & 0 & 0 & 0 & \cdots & -\lambda_2 & -\lambda_1 & 1 
\end{bmatrix} 
\begin{bmatrix} 
y_1 \cr y_2 \cr y_3 \cr y_4 \cr \vdots \cr y_T 
\end{bmatrix}
= 
\begin{bmatrix} 
\lambda_1 y_0 + \lambda_2 y_{-1} + \lambda_3 y_{-2} \cr \lambda_2 y_0 + \lambda_3 y_{-1} \cr \lambda_3 y_0 \cr 0 \cr \vdots \cr 0 
\end{bmatrix}
$$

where $y_{0}$, $y_{-1}$, and $y_{-2}$ are initial conditions.

## Exercise 6

Using the following parameters, compute the $y$ for process:

$$
y_{t} = \alpha_{0} + \alpha_{1} y_{t-1} + \alpha_{2} y_{t-2} + \alpha_{3} y_{t-3} \quad where \quad t = 1, 2 \ldots
$$

Use the following parameters:

In [ ]:
T = 100

# parameters
𝛼0 = 10.0
𝛼1 = 1.53
𝛼2 = -.9
𝛼3 = -.04

y_2 = 32. # y_{-2}
y_1 = 28. # y_{-1}
y0 = 24.

Try to visualize the process as shown in the [lecture](https://intro.quantecon.org/time_series_with_matrices.html) (Hint: you can use some insights from Exercise 5).

### Solution

In [ ]:
A = np.identity(T)  # The T x T identity matrix

for i in range(T):

    if i-1 >= 0:
        A[i, i-1] = -𝛼1

    if i-2 >= 0:
        A[i, i-2] = -𝛼2

    if i-3 >= 0:
        A[i, i-3] = -𝛼3

b = np.full(T, 𝛼0)
b[0] = 𝛼0 + 𝛼1 * y0 + 𝛼2 * y_1 + 𝛼3 * y_2
b[1] = 𝛼0 + 𝛼1 * y0 + 𝛼2 * y_1
b[2] = 𝛼0 + 𝛼2 * y0

print(A)

In [ ]:
A_inv = np.linalg.inv(A)

y = A_inv @ b

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(T)+1, y)
plt.xlabel('t')
plt.ylabel('y')

plt.show()

In [ ]:
N = 100
𝜎u = 2.
fig, ax = plt.subplots()

for i in range(N):
    col = cm.viridis(np.random.rand())  # Choose a random color from viridis
    u = np.random.normal(0, 𝜎u, size=T)
    y = A_inv @ (b + u)
    ax.plot(np.arange(T)+1, y, lw=0.5, color=col)

plt.xlabel('t')
plt.ylabel('y')

plt.show()